In [1]:
import pymysql as msql
import re

In [2]:
configs = {'host': '127.0.0.1',
           'port': 3306,
           'user': 'root',
           'password': 'xiaoyu1986',
           'db': 'znkfzl', 
           'charset': "utf8"}

In [3]:
conn = msql.connect(**configs)

# 1. `serviceorproductinfo`和`infotemp`的处理

In [10]:
with conn.cursor() as cursor:
    cursor.execute("show create table serviceorproductinfo")
    res = cursor.fetchall()

In [12]:
print(res[0][1])

CREATE TABLE `serviceorproductinfo` (
  `SERVICEORPRODUCTINFOID` decimal(22,0) DEFAULT NULL,
  `SERVICEID` decimal(22,0) DEFAULT NULL,
  `ATTR1` text CHARACTER SET utf8 COLLATE utf8_bin,
  `ATTR2` text CHARACTER SET utf8 COLLATE utf8_bin,
  `ATTR3` text CHARACTER SET utf8 COLLATE utf8_bin,
  `ATTR4` text CHARACTER SET utf8 COLLATE utf8_bin,
  `ATTR5` text CHARACTER SET utf8 COLLATE utf8_bin,
  `ATTR6` text CHARACTER SET utf8 COLLATE utf8_bin,
  `ATTR7` text CHARACTER SET utf8 COLLATE utf8_bin,
  `ATTR8` text CHARACTER SET utf8 COLLATE utf8_bin,
  `ATTR9` text CHARACTER SET utf8 COLLATE utf8_bin,
  `ATTR10` text CHARACTER SET utf8 COLLATE utf8_bin,
  `ATTR11` text CHARACTER SET utf8 COLLATE utf8_bin,
  `ATTR12` text CHARACTER SET utf8 COLLATE utf8_bin,
  `ATTR13` text CHARACTER SET utf8 COLLATE utf8_bin,
  `ATTR14` text CHARACTER SET utf8 COLLATE utf8_bin,
  `ATTR15` text CHARACTER SET utf8 COLLATE utf8_bin,
  `STATUS` decimal(22,0) DEFAULT NULL,
  `SERVICE` varchar(200) CHARACTER SET u

In [13]:
sql = res[0][1]

In [17]:
print(','.join([x for x in sql.split(',') if not re.findall('ATTR\d+', x, flags=re.I)]))

CREATE TABLE `serviceorproductinfo` (
  `SERVICEORPRODUCTINFOID` decimal(22,0) DEFAULT NULL,
  `SERVICEID` decimal(22,0) DEFAULT NULL,
  `STATUS` decimal(22,0) DEFAULT NULL,
  `SERVICE` varchar(200) CHARACTER SET utf8 COLLATE utf8_bin DEFAULT NULL,
  `ABSTRACTID` varchar(200) CHARACTER SET utf8 COLLATE utf8_bin DEFAULT NULL,
  `INSERTTIME` datetime DEFAULT NULL,
  `DOCVERSION` decimal(22,0) DEFAULT '0',
  `UPDATETIME` datetime DEFAULT NULL,
  `UPDATEAUTHORID` varchar(40) CHARACTER SET utf8 COLLATE utf8_bin DEFAULT NULL,
  `FOREIGNFLAGPATHS` text CHARACTER SET utf8 COLLATE utf8_bin,
  `TEMPLATEID` decimal(22,0) DEFAULT NULL,
  `DANGWEIID` decimal(22,0) DEFAULT NULL,
  `DANGWEIENABLE` decimal(22,0) DEFAULT NULL,
  `RELTIME` datetime DEFAULT NULL,
  `EXPERTREMINDS` text CHARACTER SET utf8 COLLATE utf8_bin,
  `RICHEDIT` text CHARACTER SET utf8 COLLATE utf8_bin,
  `DOCID` varchar(50) CHARACTER SET utf8 COLLATE utf8_bin DEFAULT NULL,
  `SERVICECLASSID` decimal(22,0) DEFAULT NULL,
  `WEIGHT` 

In [19]:
new_sql = """
CREATE TABLE kms.`serviceorproductinfo` (
  `SERVICEORPRODUCTINFOID` decimal(22,0) DEFAULT NULL,
  `SERVICEID` decimal(22,0) DEFAULT NULL,
  `STATUS` decimal(22,0) DEFAULT NULL,
  `SERVICE` varchar(200) CHARACTER SET utf8 COLLATE utf8_bin DEFAULT NULL,
  `ABSTRACTID` varchar(200) CHARACTER SET utf8 COLLATE utf8_bin DEFAULT NULL,
  `INSERTTIME` datetime DEFAULT NULL,
  `DOCVERSION` decimal(22,0) DEFAULT '0',
  `UPDATETIME` datetime DEFAULT NULL,
  `UPDATEAUTHORID` varchar(40) CHARACTER SET utf8 COLLATE utf8_bin DEFAULT NULL,
  `FOREIGNFLAGPATHS` text CHARACTER SET utf8 COLLATE utf8_bin,
  `TEMPLATEID` decimal(22,0) DEFAULT NULL,
  `DANGWEIID` decimal(22,0) DEFAULT NULL,
  `DANGWEIENABLE` decimal(22,0) DEFAULT NULL,
  `RELTIME` datetime DEFAULT NULL,
  `EXPERTREMINDS` text CHARACTER SET utf8 COLLATE utf8_bin,
  `RICHEDIT` text CHARACTER SET utf8 COLLATE utf8_bin,
  `DOCID` varchar(50) CHARACTER SET utf8 COLLATE utf8_bin DEFAULT NULL,
  `SERVICECLASSID` decimal(22,0) DEFAULT NULL,
  `WEIGHT` decimal(22,0) DEFAULT NULL,
  `WEIGHTATTENTION` varchar(100) CHARACTER SET utf8 COLLATE utf8_bin DEFAULT NULL,
  `WEIGHTTOP` varchar(100) CHARACTER SET utf8 COLLATE utf8_bin DEFAULT NULL,
  `FORMALVERSION` decimal(22,0) DEFAULT NULL,
  `UNIVERSAL` decimal(22,0) DEFAULT NULL,
  `DOCNOTEDETAILS` text CHARACTER SET utf8 COLLATE utf8_bin
) ENGINE=InnoDB DEFAULT CHARSET=utf8
"""

In [20]:
with conn.cursor() as cursor:
    cursor.execute(new_sql)
    conn.commit()

In [21]:
with conn.cursor() as cursor:
    cursor.execute('alter table kms.serviceorproductinfo add EXATTRS JSON')
    conn.commit()

## 将原serviceorproduct表中的数据转为新表

In [27]:
with conn.cursor() as cursor:
    # 获取表中的所有serviceid
    service_column = {}
    cursor.execute('select distinct serviceid from serviceorproductinfo')
    res1 = cursor.fetchall()
    print(res1)
    for x in res1:
        cursor.execute("""SELECT COLUMNNUM, `NAME`
            FROM serviceattrname2colnum
            WHERE SERVICEID=%s""", (x,))
        res2 = cursor.fetchall()
        service_column[x[0]] = {"ATTR%d"%item[0]: item[1] for item in res2}

((Decimal('1878124'),), (Decimal('1831460'),), (Decimal('1831463'),), (Decimal('1878126'),))


In [28]:
print(service_column[1878126])

{'ATTR1': 'GroupNum', 'ATTR2': 'cityId', 'ATTR3': 'cityName', 'ATTR4': 'docId', 'ATTR5': 'docName', 'ATTR6': 'foreign', 'ATTR7': 'foreignFlag', 'ATTR8': 'modifytime', 'ATTR9': 'validtime', 'ATTR217': 'FAQ.文档名', 'ATTR218': 'FAQ.用户问题', 'ATTR219': 'FAQ.有效时间', 'ATTR220': 'FAQ.在线', 'ATTR221': 'FAQ.坐席', 'ATTR222': 'FAQ.IVR'}


In [34]:
list(service_column[1878126].keys())

['ATTR1',
 'ATTR2',
 'ATTR3',
 'ATTR4',
 'ATTR5',
 'ATTR6',
 'ATTR7',
 'ATTR8',
 'ATTR9',
 'ATTR217',
 'ATTR218',
 'ATTR219',
 'ATTR220',
 'ATTR221',
 'ATTR222']

In [32]:
with conn.cursor() as cursor:
    cursor.execute('desc serviceorproductinfo')
    res = cursor.fetchall()
    common_props = ','.join([x[0] for x in res if not re.findall('^ATTR\d+$', x[0], flags=re.I)])

In [47]:
len(common_props.split(','))

24

In [49]:
import json

In [62]:
with conn.cursor() as cursor:
    cursor.execute('select distinct docid, serviceid from serviceorproductinfo')
    res = cursor.fetchall()
    print(len(res))
    i = 1
    for docid, serviceid in res:
        print(i, docid)
        colmap = service_column[int(serviceid)]
        exclusive_cols = list(colmap.keys())
        exclusive_props = ','.join(colmap.keys())
        cursor.execute('select ' + common_props + ',' + exclusive_props + ' from serviceorproductinfo where docid=%s', (docid,))
        a = cursor.fetchone()
        comm = a[:24]
        excl = a[24:]
        json_excl = {colmap[x]: excl[i] for i, x in enumerate(exclusive_cols)}
        inserted = comm + (json.dumps(json_excl), )
        cursor.execute('insert into kms.serviceorproductinfo('+ common_props + ', exattrs)' + 'values(' + ','.join(['%s']*25) + ')', inserted)
        conn.commit()
        i += 1

1b4bf6b2c74007be8daf1e
1146 cdaf5427f92a491bb6438152b1144061
1147 72abb2781e974d82aaec032e2facf216
1148 753a7ace2e8544ef888c278818d2e93a
1149 a2fa2e54187742e49e569520806e1cb4
1150 cf7c2002c6ec407fa23b56b0fb62bf12
1151 88f1a6a38ce447ada2601655414e1328
1152 6cc714cc93f7405a97c5a40c83811149
1153 01c7b7caac694dcabbe64334a84f9422
1154 517f7886746b4a93b3cc6515ef565575
1155 95ef815f8fd0437abb28e03b9caa5bd6
1156 6c24c16a4e724f89a307c17a9c4f5db1
1157 6f4f206149c14513b6a1826e0690fd44
1158 b29b54a523484f1d91fd5a69bb69d400
1159 6aed9b3ee8704d998fb753f33e27a204
1160 c47de6bbb7eb4f9eabd053405f55e23c
1161 87e8962a2ce341ee8cd96589de9d64d1
1162 83e10aec79f048c3a4c3febb6dde8ae3
1163 b40d1fbd6275416eb772227122034b98
1164 e1713cffdaeb4364bae88c9bb19ae0ab
1165 bc9160c130aa45499c191ed2355e51d1
1166 7fb57933bad145098910e854ebb14142
1167 30a54330321d49b48678dbb7264688db
1168 295e634937154bb39dff9bd8a2b00522
1169 b40cd81d49584a03aa6a9427b58ed0b2
1170 c6358f2d84224568b7674048eb2bf962
1171 73392d195dea48c2ad944b

In [36]:
','.join(service_column[1878126].keys())

'ATTR1,ATTR2,ATTR3,ATTR4,ATTR5,ATTR6,ATTR7,ATTR8,ATTR9,ATTR217,ATTR218,ATTR219,ATTR220,ATTR221,ATTR222'

In [58]:
with conn.cursor() as cursor:
    docid = 'a169774a8ddb4be38aa9b6ab7a1fef33'
    colmap = service_column[int(serviceid)]
    exclusive_cols = list(colmap.keys())
    exclusive_props = ','.join(colmap.keys())
    cursor.execute('select ' + common_props + ',' + exclusive_props + ' from serviceorproductinfo where docid=%s', (docid,))
    a = cursor.fetchone()
    comm = a[:24]
    excl = a[24:]
    json_excl = {colmap[x]: excl[i] for i, x in enumerate(exclusive_cols)}
    inserted = comm + (json.dumps(json_excl), )
    cursor.execute('insert into kms.serviceorproductinfo('+ common_props + ', exattrs)' + 'values(' + ','.join(['%s']*25) + ')', inserted)

In [59]:
conn.commit()

In [70]:
with conn.cursor() as cursor:
    cursor.execute("select * from kms.serviceorproductinfo where serviceid=1878126 limit 1")
    res = cursor.fetchone()

In [71]:
json.loads(res[-1])

{'docId': 'a169774a8ddb4be38aa9b6ab7a1fef33',
 'cityId': '010000',
 'FAQ.IVR': '您好，ETC储值卡采用实名制发行，在卡内预存通行费后，车辆通过高速公路收费站时，系统将从卡内直接扣除当次通行费。',
 'docName': '什么是储值卡',
 'foreign': '对内',
 'GroupNum': '0',
 'cityName': '部中心',
 'validtime': '2119-09-22 23:59:59',
 'FAQ.在线': '您好，ETC储值卡采用实名制发行，在卡内预存通行费后，车辆通过高速公路收费站时，系统将从卡内直接扣除当次通行费。',
 'FAQ.坐席': '您好，ETC储值卡采用实名制发行，在卡内预存通行费后，车辆通过高速公路收费站时，系统将从卡内直接扣除当次通行费。',
 'modifytime': '2019-09-22',
 'foreignFlag': '1',
 'FAQ.文档名': '什么是储值卡',
 'FAQ.有效时间': None,
 'FAQ.用户问题': '什么是储值卡'}

In [77]:
with conn.cursor(cursor=msql.cursors.DictCursor) as cursor:
    cursor.execute("select * from kmsdocument limit 1")
    res = cursor.fetchone()

In [78]:
print(res)

{'DOCCHANNEL': Decimal('10000365'), 'DOCVERSION': Decimal('0'), 'DOCTYPE': None, 'DOCTITLE': '首次办理普通护照', 'DOCSOURCE': None, 'DOCSECURITY': None, 'DOCSTATUS': Decimal('-1'), 'DOCKIND': None, 'DOCABSTRACT': None, 'DOCKEYWORDS': None, 'DOCRELWORDS': None, 'DOCPEOPLE': None, 'DOCPLACE': None, 'DOCAUTHOR': '北京采编', 'DOCEDITOR': '北京采编', 'DOCAUDITOR': None, 'DOCOUTUPID': None, 'DOCVALID': datetime.datetime(2119, 8, 31, 23, 59, 59), 'DOCPUBURL': None, 'DOCPUBTIME': None, 'DOCRELTIME': datetime.datetime(2019, 8, 6, 9, 0), 'CRUSER': '201728', 'CRTIME': datetime.datetime(2019, 8, 6, 10, 4, 50), 'DOCWORDSCOUNT': Decimal('0'), 'DOCPRO': None, 'RIGHTDEFINED': Decimal('0'), 'TITLECOLOR': None, 'TEMPLATEID': Decimal('731'), 'SCHEDULE': None, 'DOCNO': None, 'DOCFLAG': Decimal('0'), 'EDITOR': None, 'ATTRIBUTE': None, 'HITSCOUNT': Decimal('0'), 'SUBDOCTITLE': None, 'ATTACHPIC': None, 'DOCLINK': None, 'DOCFILENAME': None, 'DOCFROMVERSION': None, 'DOCPARENTID': None, 'REPLYFLAG': None, 'REPLYCOUNT': None, '